# FOSDEM'24 - A Murder Party with Lea
<center> <span style="font-size:18px">February 4, 2024</span><br>
<center> <span style="font-size:18px">by Pierre Denis</span>

## Intro: the murder...

*It was a dark, foggy night. All the occupants of Randominion Manor were sleeping peacefully. Suddenly, around 3 a.m., a horrible scream arose. Shortly after, Dr. Black is found dead in the living room. After examining the body, one discovers, clutched in the doctor's left hand, two red dice.*
<center><img src="red_dice.jpg"/></center>

*Lea Holmes, the private investigator, watches the crime scene and whispers "Mmmh... Two dice... Die+Die... Is it a rebus?*

Four suspects have been identified, with a priori proabilities to be the killer:

|![](ColMustard.jpg)     | ![](MrsPeacock.jpg) | ![](MrsWhite.jpg) | ![Plum](ProfPlum.jpg) |
|:------------------------:|:---------------------:|:-------------------:|:-----------------------:|
| Colonel Mustard: **40%**  | Mrs. Peacock: **25%**   | Mrs. White: **10%**   | Professor Plum: **25%**   |


After first inquiries, Sigmund-the-profiler declares:

> ***if** Mrs. White is the killer, <br> 
**then** she'll be absent with prob. 95 %,<br>
**otherwise** she'll be absent with prob. 20 %*
<hr>

> ***if** Mrs. Peacock is innocent,<br>
**then** she knows who's the killer with prob. 75 %*
<hr>

> ***if** Mrs. Peacock is the killer **or if** she knows who’s the killer<br>
**then** she'll be drunk with prob. 50 %,<br>
**otherwise** she'll be drunk with prob. 10 %*
<hr>

> ***if** Col. Mustard is the killer, <br> 
**then** Prof. Plum will accuse him with prob. 75 %,<br>
**otherwise** Prof. Plum will accuse him with prob. 5 %*
<hr>
How can Lea use all these uncertain information?<br>
What if Mrs. Peacok is drunk?...

## Let's meet Lea...

[**Lea**](https://bitbucket.org/piedenis/lea), in a nutshell:

* discrete probability distributions, on any Python objects
* open to various probability representations, e.g. float, fractions and symbols
* Bayesian reasonning and Probabilistic Programming (PP)
* new *exact* algorithm, based on Python's generators (["Statues algorithm"](https://arxiv.org/abs/1806.09997))
* lightweight Python module (optional sympy, pandas, numpy, matplotlib modules for advanced functions)
* open source LGPL

|           | latest Lea version  | requires           |
|-----------|---------------------|--------------------|
|**Lea 3**  | v3.4.5 (EOL)        | Python 2.6+ or 3.x |
|**Lea 4**  | v4.0.0              | Python 3.8+        |

To install Lea 4:<br>
`pip install lea==4.0.0`

or execute the following in your Jupyter Notebook (remove the `#` to comment out):

In [ ]:
# !pip install lea==4.0.0

... and few other optional packages, if needed:

In [ ]:
# --- to plot histograms --------------
# !pip install matplotlib       
# --- to make symbolic calculations ---
# !pip install sympy              

Once Lea installed, import it:

In [ ]:
import lea

In [ ]:
lea.__version__

Let's build a fair coin...

In [ ]:
coin = lea.vals("Head", "Tail")

In [ ]:
coin

*Mmmh... I prefer to see probabilities as fractions:*

In [ ]:
lea.set_prob_type("r")

In [ ]:
coin = lea.vals("Head", "Tail")
coin

Let's build now a biased coin...

In [ ]:
bcoin = lea.vals("Head", "Tail", "Tail", "Tail")
bcoin

or, equivalently, using a probability mass function (`pmf`):

In [ ]:
bcoin = lea.pmf({"Head": '1/4', "Tail": '3/4'})
bcoin

or...

In [ ]:
bcoin = lea.pmf({"Head": '1/4', "Tail": None})

or even...

In [ ]:
bcoin = lea.pmf({"Head": 1, "Tail": 3})

In [ ]:
bcoin.plot()

Let's throw the biased coin 100 times...

In [ ]:
sample = bcoin.random(100)
print(sample)

**Q**: How can I check that frequencies follow probabilities of the biased coin?<br>
**A**: Use Lea!

In [ ]:
lea.vals(*sample)

Now, let's make it more compact...

In [ ]:
"".join(bcoin[0].random(100))

In [ ]:
"".join(bcoin[0].lower().random(100))

*Observation: Python's indexing and method calls on a given probability distribution are transferred to inner values. See also use of `count` method below.*

Let's throw two biased coins:

In [ ]:
bcoin1, bcoin2 = bcoin.new(2)

`x.new(n)` creates `n` independent events, clones of `x` (same probability distribution):

In [ ]:
bcoins_joint = lea.joint(bcoin1, bcoin2)

In [ ]:
bcoins = bcoin1 + " " + bcoin2
bcoins

*Observation: Python's `+` `-` `*` `/` `<` `<=`... operators are overloaded.*

The call `a.given(b)` allows calculating *conditional probabilities* $P(A|B)$:

In [ ]:
bcoins.given(bcoin1=="Tail")

*Observation: `bcoins` is dependent of `bcoin1`. Lea does *lazy evaluation*!<br>
In Lea, this is referred as "**referential consistency**", which is enforced by the "**Statues algorithm**" (see [arXiv paper](https://arxiv.org/abs/1806.09997)).* 

In [ ]:
bcoin1.given(bcoins.startswith("H"))

*Observation: this goes both ways... `bcoins` and `bcoin1` are mutually dependent.*

Let's calculate the number of tails in `bcoins_joint`:

In [ ]:
count_tails = bcoins_joint.count("Tail")
count_tails

... and observe how `bcoins_joint` and `bcoins_joint` are interdependent:

In [ ]:
lea.joint(bcoins_joint, count_tails)

In [ ]:
bcoins_joint.given(count_tails==1)

Side note: Lea can also handle Boolean events and associated operators `&` (AND) `|` (OR) `~` (NOT):

In [ ]:
to_be = lea.event('41/42')
to_be

In [ ]:
to_be | ~to_be 

## Some funny puzzles

### One Ace in three throws 
<center><img src="TheDoctrineOfChance.png"/></center>

**<p style="text-align: center;">from "The Doctrine of Chance", by A. de Moivre, 1718</p>**

In [ ]:
die = lea.interval(1, 6)
die

In [ ]:
d1, d2, d3 = die.new(3)

In [ ]:
from lea import P

Lea's `P` function allows extracting the probability of `True`, to get the probability $P(A)$.

In [ ]:
P((d1==1) | (d2==1) | (d3==1))

Some alternative ways:

In [ ]:
P(lea.any_true(d1==1, d2==1, d3==1))

In [ ]:
P(lea.vals(1).is_any_of(d1, d2, d3))

In [ ]:
ds = lea.joint(d1, d2, d3)
nb_aces = ds.count(1)
P(nb_aces >= 1)

### Dwarf vs cave troll (RPG Combat)
<center><img src="Drong_M01.jpg"/><img src="gw-99060209197.jpg"/></center>

***<p style="text-align: center;">... At this moment, Bashful the dwarf strikes the cave troll with his magic axe. Eeeeh-aaaaaah!</p>***

In [ ]:
lea.set_prob_type("f")
lea.set_display_options(kind='%', nb_decimals=2)

In [ ]:
D6 = lea.interval(1, 6)
D20 = lea.interval(1, 20)

 Bashful's attack roll is D20+4. The trool is hit if this roll is greater than troll's armor class, which is 18.

In [ ]:
attack_roll = D20.new() + 4
troll_armor_class = 18
hit = attack_roll >= troll_armor_class
print (f"Probability that Bashful hits the troll: {(P(hit))}")

If the hit succeeds, then the damage of the magic axe is 2D6+5.

In [ ]:
damage_if_hit = D6.times(2) + 5
damage_if_hit.plot()

In [ ]:
damage = lea.if_(hit, damage_if_hit, 0)
damage.plot()

In [ ]:
print (f"Average troll damages per round: {damage.mean():5.2}")

**Assuming that the troll has 20 health points remaining, what's the probability to kill him in maximum four rounds?**

In [ ]:
troll_hp = 20
P(troll_hp - damage.times(4) <= 0)

See also [**Dice Tools**](https://github.com/hindsm38/dice_tools), a Python package using Lea dedicated to dice based tabletop games.

**ADVANCED: What is the probability distribution to kill the troll in *n* rounds (exactly)?**

nb_rounds_max = 9 

def nb_rounds_to_kill(hp, n=0):
    if n > nb_rounds_max:
        return f">{nb_rounds_max}"
    return lea.if_(hp <= 0,
                   f"{n}",
                   nb_rounds_to_kill(hp-damage.new(), n+1))

nb_rounds_to_kill_troll = nb_rounds_to_kill(troll_hp)
nb_rounds_to_kill_troll.plot("nb rounds")
print(nb_rounds_to_kill_troll)
P(nb_rounds_to_kill_troll <= "4")

### The intransitive dice 
<center><img src="Intransitive_dice.png"/></center>

* The RED die has sides 2, 2, 4, 4, 9, 9.
* The GREEN die has sides 1, 1, 6, 6, 8, 8.
* The BLUE die has sides 3, 3, 5, 5, 7, 7.


In [ ]:
lea.set_prob_type("r")
lea.set_display_options(kind='/')

In [ ]:
red_die   = lea.vals(2, 2, 4, 4, 9, 9)
green_die = lea.vals(1, 1, 6, 6, 8, 8)
blue_die  = lea.vals(3, 3, 5, 5, 7, 7)

 What are the means of these dice?

In [ ]:
[die.mean() for die in (red_die, green_die, blue_die)]

OK, the three dice have the same means. But...

In [ ]:
print (f"The RED die beats the GREEN die with prob. {P(red_die > green_die)}.")

In [ ]:
print (f"The GREEN die beats the BLUE die with prob. {P(green_die > blue_die)}.")

In [ ]:
print (f"The BLUE die beats the RED die with prob. {P(blue_die > red_die)}.")

These probabilities can be checked using the `joint` function:

In [ ]:
lea.joint(red_die>green_die, red_die, green_die)

In [ ]:
lea.joint(green_die>blue_die, green_die, blue_die)

In [ ]:
lea.joint(blue_die>red_die, blue_die, red_die)

### Boys or girls paradox
<center><img src="boy_and_girl.jpg"/></center>

See [Boy or girl paradox on Wikipedia](https://en.wikipedia.org/wiki/https://en.wikipedia.org/wiki/Boy_or_girl_paradox).

---
***The chances to be boy or girl are even.***

---

In [ ]:
child = lea.vals('boy', 'girl')
child

---
***Mr. Jones has two children. The older child is a girl.<br>
What is the probability that both children are girls?***

---

In [ ]:
youngest, eldest = child.new(2)
children = lea.joint(youngest, eldest)
children

In [ ]:
P((youngest == 'girl').given(eldest == 'girl'))

or, equivalently,

In [ ]:
nb_girls = children.count('girl')
P((nb_girls == 2).given(eldest == 'girl'))

---
***Mr. Smith has two children. At least one of them is a boy.<br>
What is the probability that both children are boys?***

---

In [ ]:
nb_boys = 2 - nb_girls
P((nb_boys == 2).given(nb_boys >= 1))

Explanation:

In [ ]:
lea.joint(eldest, youngest, nb_boys).given(nb_boys >= 1)

---
***Mrs. White has seven children. The eldest is a boy and he's got three brothers at least.<br>
What is the probability that all children are boys?***

---

In [ ]:
children = lea.joint(*child.new(7))
eldest = children[0]
nb_boys = children.count('boy')
P((nb_boys == 7).given(eldest == 'boy', nb_boys >= 4))

Explanation:

In [ ]:
lea.joint(children, nb_boys).given(eldest == 'boy', nb_boys >= 4)

For more a compact version, let's map "boy" to ♂ and "girl" to ♀...

In [ ]:
symbol_by_gender = {'boy': '♂', 'girl': '♀'}
children_symbols = children.map(lambda genders: "".join(symbol_by_gender[g] for g in genders))
lea.joint(children_symbols, nb_boys).given(eldest == 'boy', nb_boys >= 4)

### The Monty Hall problem
<center><img src="Monty_open_door.png"/></center>

See [Monty Hall problem on Wikipedia](https://en.wikipedia.org/wiki/Monty_Hall_problem).

***Suppose you're on a game show, and you're given the choice of three doors: Behind one door is a car; behind the others, goats. You pick a door, say No. 1, and the host, who knows what's behind the doors, opens another door, say No. 3, which has a goat. He then says to you, "Do you want to pick door No. 2?" Is it to your advantage to switch your choice?***

In [ ]:
door = "door " + lea.vals(*"123")
prize = door.new()
choice1 = door.new()
goat = door.such_that(door != choice1, door != prize)
lea.joint(prize, choice1, goat)

What if I keep my initial choice?

In [ ]:
P(choice1 == prize)

What if I change my choice?

In [ ]:
choice2 = door.such_that(door != choice1, door != goat)
P(choice2 == prize)

Explanation:

In [ ]:
lea.joint(prize, choice1, goat, choice2, choice2==prize).given(choice1=="door 1")

## Who's the (most probable) killer?

In [ ]:
import lea
from lea import P, pmf, if_, event
lea.set_prob_type("x")
lea.set_display_options(kind="%", nb_decimals=2, one_line=True)

Let's define the a priori probabilities to be the killer:

|![](ColMustard.jpg)     | ![](MrsPeacock.jpg) | ![](MrsWhite.jpg) | ![Plum](ProfPlum.jpg) |
|:------------------------:|:---------------------:|:-------------------:|:-----------------------:|
| Colonel Mustard: **40%**  | Mrs. Peacock: **25%**   | Mrs. White: **10%**   | Professor Plum: **25%**   |


In [ ]:
killer = pmf({ "Col. Mustard": '40 %',
               "Mrs. Peacock": '25 %',
               "Mrs. White"  : '10 %',
               "Prof. Plum"  : '25 %' })
killer

In [ ]:
killer.plot()

Now, let's write down the information provided by Sigmund-the-profiler...

> ***if** Mrs. White is the killer, <br> 
**then** she'll be absent with prob. 95 %,<br>
**otherwise** she'll be absent with prob. 20 %*
<hr>

In [ ]:
mrs_white_is_absent = if_( killer == "Mrs. White",
                           event('95 %'),
                           event('20 %'))
P(mrs_white_is_absent)

> ***if** Mrs. Peacock is innocent,<br>
**then** she knows who's the killer with prob. 75 %*
<hr>

In [ ]:
mrs_peacock_knows_killer = lea.if_( killer != "Mrs. Peacock",
                                    event(' 75 %'),
                                    event('100 %'))
P(mrs_peacock_knows_killer)

> ***if** Mrs. Peacock is the killer **or if** she knows who’s the killer<br>
**then** she'll be drunk with prob. 50 %,<br>
**otherwise** she'll be drunk with prob. 10 %*
<hr>

In [ ]:
mrs_peacock_is_drunk = lea.if_( (killer == "Mrs. Peacock") | mrs_peacock_knows_killer,
                                event('50 %'),
                                event('10 %'))
P(mrs_peacock_is_drunk)

> ***if** Col. Mustard is the killer, <br> 
**then** Prof. Plum will accuse him with prob. 75 %,<br>
**otherwise** Prof. Plum will accuse him with prob. 5 %*
<hr>

In [ ]:
prof_plum_accuses_col_mustard = if_( killer == "Col. Mustard",
                                     event("75 %"),
                                     event(" 5 %"))
P(prof_plum_accuses_col_mustard)

**Police investigation, day #1**

The day after the murder, all four suspects are called to be questioned...

In [ ]:
evidences = []
killer.given(*evidences)

**EVIDENCE 1: Mrs. White is absent.**

In [ ]:
evidences.append(mrs_white_is_absent)
killer.given(*evidences)

**EVIDENCE 2: Mrs. Peacock is drunk.**

In [ ]:
evidences.append(mrs_peacock_is_drunk)
killer.given(*evidences)

**EVIDENCE 3: Prof. Plum accuses Col. Mustard.**

In [ ]:
evidences.append(prof_plum_accuses_col_mustard)
killer.given(*evidences)

**EVIDENCE 4: The killer is a woman.**

In [ ]:
killer_is_woman = killer.startswith("Mrs.")
evidences.append(killer_is_woman)
killer.given(*evidences)

In [ ]:
P((killer == "Mrs. Peacock").given(*evidences))

In [ ]:
killer.given(*evidences).plot()

**EVIDENCE 4 REVISED: The killer is *probably* a woman (90%).**

In [ ]:
del evidences[-1]
killer.given(*evidences).given_prob(killer_is_woman, p='90 %')

In [ ]:
killer.given(*evidences).given_prob(killer_is_woman, p='90 %').plot()

In [ ]:
print(P(mrs_peacock_knows_killer.given(*evidences)))
print(P(mrs_peacock_knows_killer.given(mrs_peacock_is_drunk)))
print(P(mrs_peacock_is_drunk.given(mrs_peacock_knows_killer)))
print(P(mrs_peacock_is_drunk.given(*evidences)))

**Conclusions**<br>
OK. This is just a game... but probabilities can play a decisive role in forensic science!<br>
There are notable examples of *flaws*, notoriously the [**Dreyfus affair**](https://en.wikipedia.org/wiki/Dreyfus_affair) (1894) and the [**Sally Clark case**](https://en.wikipedia.org/wiki/Sally_Clark) (1998).<br>
See also the [***"prosecutor's fallacy"***](https://en.wikipedia.org/wiki/Base_rate_fallacy).

### Symbolic calculation (using sympy)
You may change any probability by a variable name.
For instance, for priori probabilities:

In [ ]:
lea.set_display_options(one_line=False)

In [ ]:
killer = pmf({ "Col. Mustard": 'm',
               "Mrs. Peacock": 'p',
               "Mrs. White"  : 'w',
               "Prof. Plum"  : None })
killer

...then, redo the defintion of previous rules and evidences. You'll get results as probability formulae!

**Another example:**

The probablity of win is $p$ at each trial. How many wins after 5 trials?

In [ ]:
nb_wins = lea.binom(5, 'p')
nb_wins

Given that I know that you have win at least one time, what is the probability that you've win at least 3 times?

In [ ]:
P((nb_wins >= 3).given(nb_wins >= 1))

## A bullshit generator
<center><img src="bullshit.jpg"/></center>

In [ ]:
# --- to hear the sentences (optional) ---
# !pip install pyttsx3

In [ ]:
# ======================================================================
# Bullshit Generator 
#   by Pierre Denis, March 2009, 2014, 2024
# ======================================================================

# --------------------------------------------------
# grammar engine
# --------------------------------------------------

import lea

class Node(object):

    def set_terms_choices(self,*termsChoices):
        self.terms_choices = lea.pmf(termsChoices)
        
    def gen_words(self):
        terms = self.terms_choices.random()
        for term in terms:
            if isinstance(term, str):
                yield term
            else:
                for word in term.gen_words():
                    yield word

    def get_string(self):
        res = " ".join(self.gen_words())
        res = ", ".join(w.strip() for w in res.split(",") if w.strip())
        if res.endswith(", "):
            res = res[:-2]
        return res[0].upper() + res[1:] + "."


class TerminalNode(object):

    def __init__(self,*words):
        self.words = lea.vals(*words)

    def gen_words(self):
        yield self.words.random()

# --------------------------------------------------
# grammar
# --------------------------------------------------

verb = TerminalNode(
    "accesses", "activates", "administrates", "aggregates", "builds", "calculates", "calls", "checks",
    "cleans up", "competes with", "completes", "complies with", "consumes", "controls", "covers",
    "creates", "declares", "delivers", "dispatches", "eases", "encapsulates", "encompasses",
    "executes", "extracts", "features", "generates", "gets", "gets", "gets", "governs", "guides",
    "has", "has", "has", "increases", "inherits from", "is", "is", "is", "keeps track of",
    "leverages", "lies in", "makes", "manages", "maximizes", "mitigates", "monitors", "must have",
    "needs", "negociates", "offers", "opens", "operates on", "optimizes", "orchestrates",
    "overwrites", "performs", "populates", "precludes", "promotes", "provides", "provides",
    "reads", "recalls", "receives", "reduces", "registers", "regulates", "relies on", "removes",
    "requests", "requires", "resides on", "resides within", "retrieves", "retrieves the data in",
    "runs", "runs on", "schedules", "integrates with", "sends", "shall be", "shall have", "should be",
    "should have", "starts", "stores", "streamlines", "subscribes to", "supersedes", "takes",
    "targets", "triggers", "updates", "validates", "writes")

passive_verb = TerminalNode(
    "accessed by", "achieved by", "aggregated by", "applicable for", "asserted by", "authorized by",
    "based upon", "built from", "built upon", "cleaned by", "collected by", "consumed",
    "contained in", "controlled by", "dedicated to", "deployed on", "deleted by", "derived from",
    "dispatched by", "driven by", "eased by", "enabled by", "encapsulated by", "envisioned in",
    "extracted from", "generated by", "in the scope of", "installed on", "integrated in",
    "interfaced by", "located in", "managed by", "maximized by", "monitored by", "opened by",
    "optimized by", "orchestrated by", "packaged in", "performed by", "populated by", "processed by",
    "provided by", "provided from", "received by", "recycled by", "refreshed by", "registered in",
    "removed by", "removed from", "requested by", "related to", "required by", "responsible for",
    "scheduled by", "sent to", "serialized by", "serialized in", "started in", "stored by",
    "stored in", "stored on", "subscribed by", "updated by", "validated by", "written by")

a_simple_name = TerminalNode(
    "COTS", "GRID processing", "Java program", "LDAP registry", "Portal", "RSS feed", "SAML token",
    "SOAP message", "SSO", "TCP/IP", "UDDI", "UML model", "URL", "W3C", "Web", "Web 2.0",
    "Web browser", "Web page", "Web service", "back-end", "backbone", "backend", "bandwidth", "bean",
    "box", "bridge", "browser", "bus", "business", "business model", "call", "catalogue", "class",
    "client", "cluster", "collection", "communication", "component", "compression", "computer",
    "concept", "conceptualization", "connection", "console", "content", "context", "control",
    "controller", "cookie", "copy", "customization", "data", "database", "dataset", "datastore",
    "deployment", "derivation", "design", "development", "device", "directory", "discovery",
    "dispatcher", "disruption", "document", "domain", "factory", "fat client", "feature", "file",
    "footprint", "form", "frame", "framework", "frontend", "function", "gateway", "genericity",
    "geomanagement", "goal", "governance", "granularity", "guideline", "header", "key", "layer",
    "leader", "library", "link", "list", "log file", "logic", "look-and-feel", "manager", "market",
    "mechanism", "memory", "message", "meta-model", "metadata", "model", "modeling", "module",
    "name", "network", "package", "packaging", "parallelism", "performance", "persistence",
    "personalization", "plug-in", "policy", "port", "portal", "practice", "presentation layer",
    "printer", "privacy", "private key", "procedure", "process", "processor", "processing",
    "product", "protocol", "provider", "recommendation", "registration", "registry", "relationship",
    "request", "resource", "responsibility", "role", "rule", "scenario", "scheduler", "schema",
    "security", "sequence", "server", "service", "service provider", "servlet", "session",
    "skeleton", "software", "solution", "source", "space", "specification", "suite", "signal",
    "slot", "standard", "state", "statement", "streaming", "style sheet", "subscriber", "subsystem",
    "system", "system", "table", "target", "task", "taxonomy", "technique", "technology", "template",
    "thin client", "thread", "throughput", "time", "timing", "token", "tool", "toolkit", "topic",
    "unit", "usage", "use case", "user", "user experience", "validation", "value", "version",
    "vision", "warehouse", "work", "workflow", "zone")

an_simple_name = TerminalNode(
    "API", "IP address", "Internet", "XML", "abstraction", "access", "acknowledgment", "action",
    "actor", "administrator", "aggregator", "algorithm", "allowance", "appliance", "application",
    "approach", "architecture", "area", "artifact", "aspect", "authentication", "automation",
    "availability", "encapsulation", "end-point", "engine", "entity", "environment", "event",
    "identifier", "image", "information", "instance", "instantiation", "integration", "interface",
    "interoperability", "issuer", "object", "ontology", "operation", "operator", "opportunity",
    "option", "orchestration", "order", "owner")

a_adjective = TerminalNode(
    "BPEL",  "DOM", "DTD", "GRID", "HTML", "J2EE", "Java", "Java-based", "UML", "SAX", "WFS", "WSDL",
    "basic", "broad", "bug-free", "business-driven", "client-side", "coarse", "coherent",
    "compatible", "complete", "compliant", "comprehensive", "conceptual", "consistent", "continuous", 
    "cost-effective", "custom", "data-driven", "dedicated", "design", "disruptive", "distributed",
    "dynamic", "encrypted", "event-driven", "fine-grained", "first-class", "form", "formal", "free",
    "full", "generic", "geo-referenced", "global", "global", "graphical", "hard", "high-resolution",
    "high-level", "individual", "invulnerable", "just-in-time", "key", "layered", "leading",
    "lightweight", "limited", "local", "logical", "machine", "main", "major", "message-based",
    "most important", "multi-tiers", "narrow", "native", "next", "next-generation", "normal",
    "operational", "parallel", "password-protected", "peer-to-peer", "performant", "periodic",
    "physical", "point-to-point", "polymorphic","portable", "primary", "prime", "private", "proven",
    "public", "raw", "real-time", "registered", "relational", "reliable", "remote", "respective",
    "right", "robust", "rule-based", "scalable", "seamless", "secondary", "semantic", "serial",
    "server-side", "service-based", "service-oriented", "simple", "sole", "specific", "sporadic",
    "standard", "state-of-the-art", "stateless", "storage", "sufficient", "technical", "thread-safe",
    "time-based", "uniform", "unique", "used", "useful", "user-friendly", "virtual", "visual",
    "Web-based", "web-centric", "well-documented", "wireless", "world-leading", "zero-default")

an_adjective = TerminalNode(
    "AJAX", "OO", "XML-based", "abstract", "agnostic", "ancillary", "asynchronous",
    "authenticated", "authorized", "auto-regulated", "automated", "automatic", "available", "aware",
    "efficient", "event-based", "integrated", "international", "interoperable", "off-line",
    "off-the-shelf", "official", "online", "open", "operational", "other", "own", "unaffected",
    "unlimited", "up-to-date")

adverb = TerminalNode(
    "basically", "comprehensively", "conceptually", "consistently", "definitely", "dramatically",
    "dynamically", "expectedly", "fully", "generally", "generically", "globally", "greatly",
    "individually", "locally", "logically", "mainly", "mostly", "natively", "officially", "often",
    "periodically", "physically", "practically", "primarily", "repeatedly", "roughly", "sequentially",
    "simply", "specifically", "surely", "technically", "undoubtly", "usefully", "virtually")
                            
sentenceHead = TerminalNode(
    "actually", "as a matter of fact", "as said before", "as stated before", "basically",
    "before all", "besides this", "beyond that point", "clearly", "conversely", "despite these facts",
    "for this reason", "generally speaking", "if needed", "in essence", "in other words",
    "in our opinion", "in the long term", "in the short term", "in this case", "incidentally",
    "moreover", "nevertheless", "now", "otherwise", "periodically", "roughly speaking",
    "that being said", "then", "therefore", "to summarize", "up to here", "up to now",
    "when this happens")

(name, a_name, an_name, name_tail, adjective, name_group,
 simple_name_group, verbal_group, simple_verbal_group, sentence,
 sentence_tail) = [Node() for i in range(11)]

a_name.set_terms_choices(
    (( a_simple_name,      ), 50 ),
    (( a_simple_name, name ),  8 ),
    (( a_name, name_tail   ),  5 ))

an_name.set_terms_choices(
    (( an_simple_name,      ), 50 ),
    (( an_simple_name, name ),  8 ),
    (( an_name, name_tail   ),  5 ))

name_tail.set_terms_choices(
    (( "of", name_group        ), 8 ),
    (( "from", name_group      ), 8 ),
    (( "above", name_group     ), 1 ),
    (( "after", name_group     ), 1 ),
    (( "against", name_group   ), 1 ),
    (( "before", name_group    ), 1 ),
    (( "behind", name_group    ), 1 ),
    (( "below", name_group     ), 1 ),
    (( "on top of", name_group ), 1 ),
    (( "under", name_group     ), 1 ))

name.set_terms_choices(
    (( a_name,  ), 1 ),
    (( an_name, ), 1 ))

adjective.set_terms_choices(
    (( a_adjective,  ), 1 ),
    (( an_adjective, ), 1 ))

name_group.set_terms_choices(
    (( simple_name_group,                                     ), 14 ),
    (( simple_name_group, passive_verb, name_group            ),  3 ),
    (( simple_name_group, "that", simple_verbal_group         ),  2 ),
    (( simple_name_group, ", which", simple_verbal_group, "," ),  1 ))

simple_name_group.set_terms_choices(
    (( "the", name              ), 40 ),
    (( "the", adjective, name   ), 20 ),
    (( "a", a_name              ), 10 ),
    (( "an", an_name            ), 10 ),
    (( "a", a_adjective, name   ),  5 ),                
    (( "an", an_adjective, name ),  5 ))  

verbal_group.set_terms_choices(
    (( verb, name_group                       ), 10 ),
    (( adverb, verb, name_group               ),  1 ),
    (( "is", passive_verb, name_group         ),  4 ),
    (( "is", adverb, passive_verb, name_group ),  1 ),
    (( "is", adjective                        ),  1 ),
    (( "is", adverb, adjective                ),  1 ))

simple_verbal_group.set_terms_choices(
    (( verb, simple_name_group ), 2 ),
    (( "is", adjective         ), 1 ))

sentence.set_terms_choices(
    (( name_group, verbal_group                     ), 20 ),
    (( sentenceHead, "," , name_group, verbal_group ),  4 ),
    (( sentence, sentence_tail                      ),  4 ))

sentence_tail.set_terms_choices(
    (( "in", name_group                  ), 12 ),
    (( "within", name_group              ),  5 ),
    (( "where", name_group, verbal_group ),  5 ),
    (( "when", name_group, verbal_group  ),  5 ),
    (( "because it", verbal_group        ),  2 ),
    (( "; that's why it", verbal_group   ),  1 ))

# --------------------------------------------------
# main program
# --------------------------------------------------
try:
    import pyttsx3
    from pyttsx3 import speak
    engine = pyttsx3.init()
    engine.setProperty('rate', 120)
except ImportError:
    def speak(sentence):
        pass

try:
    from IPython.display import display, Markdown
    def show(sentence):
        display(Markdown(f'<span style="font-size:20px">{sentence}</span>'))
except ImportError:
    show = print
    
from random import seed
seed(666)
while True:
    print ("")
    for i in range(4):
        generated_sentence = sentence.get_string()
        show(generated_sentence)
        speak(generated_sentence)
    #print("\n")
    cmd = input("Press enter if you want that I continue. Press 'q', then enter to stop.")
    if cmd.strip().lower() == "q":
        break


## Q & A

In [ ]:
lea.vals("Any", "No") + " question?"

In [ ]:
lea.if_(to_be,
        lea.vals("Any", "No") + " question?",
        "What?")

## References
Lea Git repo: http://bitbucket.org/piedenis/lea<br>
contact: pie.denis@skynet.be<br>
English / French spoken!